# AI Urban Warfare Analyst - Stage 3: Performance Summary & Infographic

This notebook demonstrates **Stage 3** of the Urban Warfare Analyst pipeline:

1. **Load frame analyses** from Stage 2
2. **Aggregate statistics** across all frames
3. **Generate performance summary** with ratings and recommendations
4. **Create tactical infographic** using Gemini 3 Pro Image (top-down view)
5. **Export reports** in JSON and human-readable formats

**Target Runtime:** ~60 seconds (including infographic generation)

---

## Cell 1: Environment Setup and Imports

**Expected Runtime:** ~5 seconds

In [1]:
# Install dependencies if needed (uncomment for Google Colab)
# !pip install -q google-generativeai Pillow opencv-python python-dotenv

import sys
import json
from pathlib import Path
from IPython.display import Image as IPImage, display, HTML
import ipywidgets as widgets

# Import our custom modules
from config import Config, config
from performance_summarizer import PerformanceSummarizer

# Initialize configuration
print("🚀 AI URBAN WARFARE ANALYST - STAGE 3")
print("=" * 60)
config.print_config()
print()

# Initialize summarizer
summarizer = PerformanceSummarizer(config)

print("✅ Performance summarizer initialized")
print(f"🎨 Image Model: {config.GEMINI_IMAGE_MODEL}")
print(f"💾 Summary Output: {config.SUMMARY_DIR}")
print(f"🖼️  Infographic Output: {config.INFOGRAPHIC_DIR}")
print("\n" + "=" * 60)

🚀 AI URBAN WARFARE ANALYST - STAGE 3
AI URBAN WARFARE ANALYST - CONFIGURATION
API Key Set: ✅

Models:
  Stage 2 (Analysis): gemini-2.5-flash-preview-09-2025
  Stage 3 (Overlays): gemini-3-pro-image-preview
  Default/Fallback: gemini-2.5-flash-preview-09-2025

Model Parameters:
  Temperature: 0.7
  Max Tokens: 2048

Frame Extraction:
  Number of Frames: 3
  Extraction Positions: ['25%', '50%', '75%']
  Max Frame Size: 720px
  Frame Format: JPEG (Quality: 85)

Video Limits:
  Max Duration: 120s
  Max File Size: 100MB
  Supported Formats: .mp4, .mov, .avi, .mkv, .wmv

Directories:
  Base: /Users/samuel.tan/Desktop/Other Projects/2025 Q4/Army AI Transformation/Urban Warfare Analyst
  Output: /Users/samuel.tan/Desktop/Other Projects/2025 Q4/Army AI Transformation/Urban Warfare Analyst/outputs
  Prompts: /Users/samuel.tan/Desktop/Other Projects/2025 Q4/Army AI Transformation/Urban Warfare Analyst/prompts

✅ Performance summarizer initialized
🎨 Image Model: gemini-3-pro-image-preview
💾 Summar

## Cell 2: Load Analysis Results from Stage 2

**Expected Runtime:** ~2 seconds

Load the frame analyses generated in Stage 2

In [2]:
# 📝 EDIT THIS TO MATCH YOUR VIDEO NAME
video_name = "urban_warfare_training"

print(f"📁 Loading analysis results for: {video_name}")
print("=" * 60)

# Load analysis JSON files
analysis_dir = config.ANALYSIS_JSON_DIR / video_name

if not analysis_dir.exists():
    raise FileNotFoundError(f"Analysis directory not found: {analysis_dir}")

analysis_files = sorted(analysis_dir.glob("analysis_*.json"))

if not analysis_files:
    raise FileNotFoundError(f"No analysis files found in: {analysis_dir}")

print(f"✅ Found {len(analysis_files)} analysis files")

# Load each analysis
analysis_results = []
for i, file_path in enumerate(analysis_files):
    with open(file_path, 'r') as f:
        analysis = json.load(f)
        
    analysis_results.append({
        'index': i,
        'timestamp': analysis.get('timestamp', 0),
        'analysis': analysis,
        'json_path': file_path
    })
    
    print(f"  📄 Loaded: {file_path.name} (Score: {analysis.get('score', 'N/A')}/100)")

# Load video metadata (reconstruct from first analysis)
video_metadata = {
    'filename': f"{video_name}.mp4",
    'duration': max(r['timestamp'] for r in analysis_results) * 1.33,  # Approximate
    'resolution': [1280, 720]  # Default, adjust if known
}

print(f"\n✅ Loaded {len(analysis_results)} frame analyses")
print(f"📊 Video: {video_metadata['filename']}")
print(f"⏱️  Duration: ~{video_metadata['duration']:.1f}s")
print("\n" + "=" * 60)

📁 Loading analysis results for: urban_warfare_training
✅ Found 3 analysis files
  📄 Loaded: analysis_000_9.4s.json (Score: 85/100)
  📄 Loaded: analysis_001_18.9s.json (Score: 91/100)
  📄 Loaded: analysis_002_28.3s.json (Score: 62/100)

✅ Loaded 3 frame analyses
📊 Video: urban_warfare_training.mp4
⏱️  Duration: ~37.6s



## Cell 3: Aggregate Analysis

**Expected Runtime:** ~5 seconds

Aggregate statistics from all frame analyses

In [3]:
print("📊 AGGREGATING FRAME ANALYSES")
print("=" * 60)

# Aggregate analysis
summary = summarizer.aggregate_analysis(
    analysis_results=analysis_results,
    video_metadata=video_metadata
)

print("\n" + "=" * 60)
print("✅ Aggregation complete!")
print("=" * 60)

📊 AGGREGATING FRAME ANALYSES
📊 Aggregating analysis from all frames...
✅ Aggregation complete:
   Average Score: 79.3/100
   Rating: Good
   Total Errors: 6
   Total Strengths: 9

✅ Aggregation complete!


## Cell 4: Display Performance Summary

View the aggregated performance metrics

In [4]:
print("🎉 PIPELINE COMPLETE: ALL 3 STAGES FINISHED")
print("=" * 80)

# Format timestamps
timestamps_str = [f"{r['timestamp']:.1f}s" for r in analysis_results]

print("\n✅ STAGE 1: Frame Extraction")
print(f"  - Extracted {summary['frame_count']} frames from video")
print(f"  - Timestamps: {timestamps_str}")

print("\n✅ STAGE 2: Frame Analysis & Annotation")
print(f"  - Analyzed {summary['frame_count']} frames with Gemini 2.5 Flash")
print(f"  - Generated {summary['frame_count']} annotated frames")
print(f"  - Identified {summary['tactical_errors']['total']} errors, {summary['tactical_strengths']['total']} strengths")

print("\n✅ STAGE 3: Performance Summary")
print(f"  - Average Score: {summary['overall_performance']['average_score']}/100")
print(f"  - Rating: {summary['overall_performance']['rating']}")
print(f"  - Generated {len(summary['recommendations'])} recommendations")

print("\n📂 OUTPUT FILES:")
print(f"  Stage 1: outputs/urban_warfare_training/frames/ ({summary['frame_count']} images)")
print(f"  Stage 2:")
print(f"    - outputs/analysis_json/{video_name}/ ({summary['frame_count']} JSON files)")
print(f"    - outputs/annotated_frames/{video_name}/ ({summary['frame_count']} images)")
print(f"  Stage 3:")
print(f"    - {config.SUMMARY_DIR / f'summary_{video_name}.json'}")
print(f"    - {config.SUMMARY_DIR / f'report_{video_name}.txt'}")
if infographic:
    print(f"    - {config.INFOGRAPHIC_DIR / f'infographic_{video_name}.png'}")

print("\n" + "=" * 80)
print("✨ Urban Warfare Analyst - Analysis Complete!")
print("=" * 80)

# Display final summary card
print("\n" + "┌" + "─" * 78 + "┐")
print(f"│  VIDEO: {summary['video_name']:^68} │")
print("├" + "─" * 78 + "┤")
avg_score_str = f"{summary['overall_performance']['average_score']:.1f}/100"
rating_str = f"({summary['overall_performance']['rating']})"
print(f"│  Overall Score: {avg_score_str} {rating_str:^45} │")
print(f"│  Frames: {summary['frame_count']}    Soldiers: {summary['total_soldiers']}    Errors: {summary['tactical_errors']['total']}    Strengths: {summary['tactical_strengths']['total']}                       │")
print("└" + "─" * 78 + "┘")

🎉 PIPELINE COMPLETE: ALL 3 STAGES FINISHED

✅ STAGE 1: Frame Extraction
  - Extracted 3 frames from video
  - Timestamps: ['9.4s', '18.9s', '28.3s']

✅ STAGE 2: Frame Analysis & Annotation
  - Analyzed 3 frames with Gemini 2.5 Flash
  - Generated 3 annotated frames
  - Identified 6 errors, 9 strengths

✅ STAGE 3: Performance Summary
  - Average Score: 79.3/100
  - Rating: Good
  - Generated 3 recommendations

📂 OUTPUT FILES:
  Stage 1: outputs/urban_warfare_training/frames/ (3 images)
  Stage 2:
    - outputs/analysis_json/urban_warfare_training/ (3 JSON files)
    - outputs/annotated_frames/urban_warfare_training/ (3 images)
  Stage 3:
    - /Users/samuel.tan/Desktop/Other Projects/2025 Q4/Army AI Transformation/Urban Warfare Analyst/outputs/summaries/summary_urban_warfare_training.json
    - /Users/samuel.tan/Desktop/Other Projects/2025 Q4/Army AI Transformation/Urban Warfare Analyst/outputs/summaries/report_urban_warfare_training.txt


NameError: name 'infographic' is not defined

## Cell 5: Generate Tactical Infographic

**Expected Runtime:** ~30-60 seconds

Generate top-down tactical view infographic using Gemini 3 Pro Image

In [5]:
print("🎨 TACTICAL INFOGRAPHIC GENERATION")
print("=" * 60)

# Generate infographic
infographic = summarizer.generate_infographic(
    summary=summary,
    analysis_results=analysis_results
)

if infographic:
    print("\n✅ Infographic generated!")
    
    # Save infographic
    infographic_path = config.INFOGRAPHIC_DIR / f"infographic_{video_name}.png"
    infographic.save(infographic_path)
    print(f"💾 Saved to: {infographic_path}")
    
    # Display infographic
    print("\n📊 Displaying Tactical Infographic:")
    display(infographic)
else:
    print("\n⚠️  Infographic generation not available (API limitation)")
    print("💡 Future enhancement: Gemini 3 Pro Image will generate tactical diagrams")
    print("\n📝 Infographic would include:")
    print("  - Top-down view of each frame")
    print("  - Green zones: Covered areas")
    print("  - Red zones: Blindspots/danger areas")
    print("  - Soldier positions with numbered icons")
    print("  - Coverage arcs showing fields of fire")
    print("  - Frame scores and formations")

print("\n" + "=" * 60)

🎨 TACTICAL INFOGRAPHIC GENERATION
🎨 Generating tactical infographic...
  📡 Calling Gemini 3 Pro Image model...
  ⚠️  Note: Image generation may take 30-60 seconds...
  ⚠️  Gemini 3 Pro Image generation not yet available via API
  💡 Future enhancement: Will generate tactical infographic automatically

⚠️  Infographic generation not available (API limitation)
💡 Future enhancement: Gemini 3 Pro Image will generate tactical diagrams

📝 Infographic would include:
  - Top-down view of each frame
  - Green zones: Covered areas
  - Red zones: Blindspots/danger areas
  - Soldier positions with numbered icons
  - Coverage arcs showing fields of fire
  - Frame scores and formations



## Cell 6: Save Summary Reports

Export summary in JSON and text formats

In [6]:
print("💾 SAVING SUMMARY REPORTS")
print("=" * 60)

# Save JSON summary
json_path = summarizer.save_summary_json(
    summary=summary,
    video_name=video_name
)
print(f"✅ JSON saved: {json_path}")

# Save text report
report_path = summarizer.save_summary_report(
    summary=summary,
    video_name=video_name
)
print(f"✅ Report saved: {report_path}")

print("\n" + "=" * 60)
print("✅ All reports saved successfully!")
print("=" * 60)

💾 SAVING SUMMARY REPORTS
💾 Summary saved: /Users/samuel.tan/Desktop/Other Projects/2025 Q4/Army AI Transformation/Urban Warfare Analyst/outputs/summaries/summary_urban_warfare_training.json
✅ JSON saved: /Users/samuel.tan/Desktop/Other Projects/2025 Q4/Army AI Transformation/Urban Warfare Analyst/outputs/summaries/summary_urban_warfare_training.json
📄 Report saved: /Users/samuel.tan/Desktop/Other Projects/2025 Q4/Army AI Transformation/Urban Warfare Analyst/outputs/summaries/report_urban_warfare_training.txt
✅ Report saved: /Users/samuel.tan/Desktop/Other Projects/2025 Q4/Army AI Transformation/Urban Warfare Analyst/outputs/summaries/report_urban_warfare_training.txt

✅ All reports saved successfully!


## Cell 7: Display Text Report

View the complete human-readable report

In [7]:
print("📄 TACTICAL PERFORMANCE REPORT")
print("\n")

# Read and display the report
with open(report_path, 'r') as f:
    report_content = f.read()

print(report_content)

📄 TACTICAL PERFORMANCE REPORT


TACTICAL PERFORMANCE SUMMARY

Video: urban_warfare_training.mp4
Duration: 37.6 seconds
Frames Analyzed: 3
Total Soldiers: 11

OVERALL PERFORMANCE

Average Score: 79.3/100
Rating: Good
Score Range: 62-91

FRAME-BY-FRAME SCORES

Frame 1 (T=9.4s): 85/100 - COLUMN

Frame 2 (T=18.9s): 91/100 - SPLIT-STACK

Frame 3 (T=28.3s): 62/100 - SPLIT-STACK

COVER UTILIZATION

Full Cover: 0 instances
Partial Cover: 6 instances
No Cover: 2 instances
Exposed: 5 instances

TACTICAL ERRORS (6 total)

1. Soldiers 2 and 3 are standing upright, presenting a larger profile than necessary while stacked, increasing vulnerability to fire or fragmentation upon entry.

2. Soldier 1 appears to be leaning slightly too far into the corner, risking over-exposure before the decision to breach is made.

3. The trailing elements (Soldiers 3 and 4) appear slightly too close to the point men, potentially compressing the stack depth and making the approach vulnerable to area denial fire.

4. T

## Cell 8: Stage 3 Complete

Pipeline complete - all stages finished!

In [8]:
print("🎉 PIPELINE COMPLETE: ALL 3 STAGES FINISHED")
print("=" * 80)

print("\n✅ STAGE 1: Frame Extraction")
print(f"  - Extracted {summary['frame_count']} frames from video")
print(f"  - Timestamps: {[f\"{r['timestamp']:.1f}s\" for r in analysis_results]}")

print("\n✅ STAGE 2: Frame Analysis & Annotation")
print(f"  - Analyzed {summary['frame_count']} frames with Gemini 2.5 Flash")
print(f"  - Generated {summary['frame_count']} annotated frames")
print(f"  - Identified {summary['tactical_errors']['total']} errors, {summary['tactical_strengths']['total']} strengths")

print("\n✅ STAGE 3: Performance Summary")
print(f"  - Average Score: {summary['overall_performance']['average_score']}/100")
print(f"  - Rating: {summary['overall_performance']['rating']}")
print(f"  - Generated {len(summary['recommendations'])} recommendations")

print("\n📂 OUTPUT FILES:")
print(f"  Stage 1: outputs/urban_warfare_training/frames/ ({summary['frame_count']} images)")
print(f"  Stage 2:")
print(f"    - outputs/analysis_json/{video_name}/ ({summary['frame_count']} JSON files)")
print(f"    - outputs/annotated_frames/{video_name}/ ({summary['frame_count']} images)")
print(f"  Stage 3:")
print(f"    - {json_path}")
print(f"    - {report_path}")
if infographic:
    print(f"    - {config.INFOGRAPHIC_DIR / f'infographic_{video_name}.png'}")

print("\n" + "=" * 80)
print("✨ Urban Warfare Analyst - Analysis Complete!")
print("=" * 80)

# Display final summary card
print("\n" + "┌" + "─" * 78 + "┐")
print(f"│  VIDEO: {summary['video_name']:^68} │")
print("├" + "─" * 78 + "┤")
print(f"│  Overall Score: {summary['overall_performance']['average_score']:.1f}/100 ({summary['overall_performance']['rating']:^30}) │")
print(f"│  Frames: {summary['frame_count']}    Soldiers: {summary['total_soldiers']}    Errors: {summary['tactical_errors']['total']}    Strengths: {summary['tactical_strengths']['total']:>17} │")
print("└" + "─" * 78 + "┘")

SyntaxError: f-string expression part cannot include a backslash (1052065226.py, line 6)

---

## Implementation Notes

### Stage 3 Features:
- **Performance Aggregation**: Combines data from all analyzed frames
- **Statistical Analysis**: Calculates averages, ranges, and trends
- **Category Scoring**: Evaluates cover utilization, formation discipline, etc.
- **Tactical Infographic**: Top-down view showing coverage zones (future enhancement)
- **Training Recommendations**: AI-generated actionable feedback
- **Multiple Export Formats**: JSON (machine-readable) and TXT (human-readable)

### Infographic Concept:
When Gemini 3 Pro Image API supports generation, it will create:
- **Top-down room layout** for each frame
- **Green zones**: Areas covered by soldiers' lines of fire
- **Red zones**: Blindspots and danger areas
- **Soldier positions**: Numbered tactical icons
- **Coverage arcs**: Visual representation of fields of fire
- **Professional styling**: Military briefing quality

### Performance Ratings:
- **90-100**: Excellent
- **75-89**: Good
- **60-74**: Satisfactory
- **50-59**: Needs Improvement
- **0-49**: Requires Remedial Training

### Success Criteria: ✅
- [x] Load analyses from Stage 2
- [x] Aggregate frame statistics
- [x] Calculate overall performance metrics
- [x] Generate training recommendations
- [x] Prepare data for infographic generation
- [x] Export JSON and text reports
- [x] Display comprehensive summary